In [ ]:
'''
ssg convert process용 임시
-ep download
-convert to feed
-feed upload

*대용량 카탈로그의 경우
-피드 제품수 제한 (900만건 이상에서 오류남..) 
-facebook api 업로드 방식은 truncate insert / update only 두가지
-피드 나누어 제품 업로드해야함
-증분 업데이트시 개별 피드에 모두 해줘야함

*일단구현
-feed write시 
-카탈로그에 나눠진 피드 upload
-증분 업데이트 추가해야함
-증분 업데이트는 피드별로 모두 upload(update_only)

*ssg의 경우
-카탈로그1 -> 카탈로그2 수정만해서 재배포 (링크값)
-1,2 카탈로그 증분 업데이트
-머천센터 업로드는 500만건으로 고정되어있음(확인?)
-custom 링크/제외카테고리/이미지사이즈
'''

## Import

In [9]:
import pandas as pd
import os
import gc
import zipfile
import requests
import asyncio, aiohttp, aiofiles
from starlette.config import Config
from urllib import parse
import json, time, datetime

## config

In [36]:
# config = {    
#     "info" : {
#         "media" : "facebook",
#         "catalog_id" : "268046537186348",
#         "feed_id" : {"2499714026735797","457448645680512"}        
#         "name" : "ssg_ep_test"
#     },
#     "ep" : {
#         "url" : "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/test/data/ssg_10g.csv",
#         "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_268046537186348.csv",
#         "format" : "csv",
#         "sep" : ",",
#         "encoding" : "utf-16"        
#     },
#     "feed" : {
#         "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_268046537186348.tsv",        
#     },
#     "columns" : {
#         "id" : "id",
#         "title" : "title",
#         "link" : "link",
#         "image_link" : "image_link",
#         "price" : "value",
#         "brand" : "brand",
#         "description" : "description",
#         "availability" : "availability",
#         "condition" : "condition"
#     }    
# }

In [90]:
config = {    
    "info" : {
        "media" : "facebook",        
        "catalog_id" : "432089861336082",
        "feed_id" : ["236164118048821","254210579552181"],
        "name" : "hellonature_ep_test"
    },
    "ep" : {        
        "url" : "http://ep.hellonature.co.kr/PGD/EP/NAVER/all.txt",
        "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_432089861336082.csv",
        "format" : "tsv",
        "sep" : "\t",
        "encoding" : "utf-8"
    },
    "feed" : {        
        "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_432089861336082.tsv",
    },
    "columns" : {
         "id" : "id",
        "title" : "title",
        "link" : "link",
        "image_link" : "image_link",
        "price_pc" : "value",
        "brand" : "brand"
    }    
}

## Util

In [26]:
async def epDownload(fromUrl, toPath):
    async with aiohttp.ClientSession() as session:
        async with session.get(fromUrl) as response:
            chunk_size = 1024*1024*10 # 10MB
            async with aiofiles.open(toPath, 'wb') as f:
                while True:
                    chunk = await response.content.read(chunk_size)                        
                    if not chunk : break
                    await f.write(chunk)

                # result = self.getInfo(toPath)
                # self.logger.info('Download complete '+str(result))
                # return result

In [16]:
def epLoad():
    # 원본 컬럼리스트 확인
    columns = pd.read_csv(
        config['ep']['path'], 
        nrows=1, sep=config['ep']['sep'], 
        encoding=config['ep']['encoding']
        )
    columns = list(columns)

    result = pd.read_csv(
        config['ep']['path'],
        nrows=None,
        chunksize=1000,
        header=0,
        dtype=str,
        sep=config['ep']['sep'],
        error_bad_lines=False,
        usecols=columns,
        encoding=config['ep']['encoding']
        )
    return result


In [97]:
async def feedWrite(num, feedPath, df):
    # print(feedPath)
    if num == 0:
        mode='w' # 새로쓰기
        header=True 
    else:
        mode='a' # 이어쓰기
        header=False
        
    df.to_csv(feedPath, 
                index=False, # 자체 인덱스제거
                sep='\t', 
                mode=mode,
                header=header, # 컬럼명 
                encoding='utf-8')

In [25]:
async def facebookUpload( feed_id, feed_url, isUpdateOnly='false'):
    api_url = f'https://graph.facebook.com/v9.0/{feed_id}/uploads'
    params ={'update_only': isUpdateOnly ,'access_token': self.access_token, 'url': feed_url}       
    async with aiohttp.ClientSession() as session:
        async with session.post(api_url, data=params) as response:
            result = await response.text()
            self.logger.info('Result '+result)
            return result 

In [70]:
def convertFilter(dataFrame):
    result = dataFrame

    keys = list(config['columns'].keys())
    result = result[keys]
    result.rename(columns=config['columns'], inplace=False)
    
    # 공백제거
    for key in keys:
        result[key] = result[key].str.strip()

    # 기본값
    if 'availability' not in result :
        result['availability'] = 'in stock'
    if 'condition' not in result : 
        result['condition'] = 'new'
    if 'description' not in result :
        result['description'] = result['title']

    ########################
    ## custom filter 작성 ##
    ########################
    
    # SSG_EPPE / SSG_EPPE_IOS
    if config['info']['catalog_id'] == '268046537186348' or config['info']['catalog_id'] == '225456985373646': 
        # link값 변경
        item_id = result['id']
        result['link'] = f'https://ad.adpool.co.kr/app/ssg/item/{item_id}'
        
        # 제외 title
        stopword = ['마스크','소독제','손소독겔','새니타이저','손살균제','snitizers','disinfection wipes']
        result = result[~result['title'].str.contains(stopword)]

        # 이미지사이즈 조정?
        result['image_link'] = result['image_link'].str.replace('_500','_600', regex=False)

    elif config['info']['catalog_id'] == '':
        pass

    return result

        




## Convert Process

In [98]:
'''
대용량파일의 경우 분할하여 저장..(카탈로그 내 피드가 여러개인 경우)
'''

async def execute():
    # ep download
    await epDownload(config['ep']['url'], config['ep']['path'])

    # total
    totalCount = 0
    totalChunkCount = 0
    for num, chunkDF in enumerate(epLoad()):
        totalCount = totalCount + len(chunkDF)
        totalChunkCount = num

    # segmentation
    feedCount = len(config['info']['feed_id']) # 카탈로그당 나눈 피드갯수
    chunkCountPerFeed = int(totalChunkCount/feedCount) # 피드하나당 chunk갯수
    print(f'chunkCountPerFeed - {chunkCountPerFeed}')


    # chunk read
    count = 0
    feedFileIndex=0
    for num, chunkDF in enumerate(epLoad()):
        count = count + len(chunkDF)

        # filter
        chunkDF = convertFilter(chunkDF)

        # write
        segmentIndex = num % chunkCountPerFeed # feed_id당 배분될 chunk만큼의 index 생성
        if segmentIndex == 0: # 0일때마다 feed 분할
            feedSecPath = str(config['feed']['path']).replace('.tsv', f'_{feedFileIndex}.tsv')
            feedFileIndex = feedFileIndex + 1
    
    
        await feedWrite(segmentIndex, feedSecPath, chunkDF)

        del[[chunkDF]]
        gc.collect()

        # break


    # facebook api upload

In [99]:
await execute()

chunkCountPerFeed - 1


In [31]:
await epDownload(config['ep']['url'], config['ep']['path'])